In [ ]:
!pip install konlpy
!pip install pyLDAvis
!pip install wordcloud

In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

# 형태소 분석기
#from konlpy.tag import Mecab
#mecab = Mecab()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식
from gensim import corpora # 단어 빈도수 계산
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용
from collections import Counter # 단어 등장 횟수 카운트

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('train_hakaton.csv')
test_df = pd.read_csv('test_hakaton.csv')
train_df

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."
...,...,...,...,...
54604,TRAIN_54604,국제,"용인 아파트서 30대 여성, 아들 딸 함께 추락 ""극단 선택 추정""","용인,아파트,여성,아들,추락,극단,선택,추정,경찰,현장,유서,경위,조사,처인구,경기..."
54605,TRAIN_54605,사회:교육_시험,"용인시, '위탁부모 보수교육' 실시","용인시,위탁,부모,보수,교육,용인시,경기,남부,가정,위탁,지원,센터,시청,컨퍼런스,..."
54606,TRAIN_54606,지역,"용인시, '플랫폼 시티' 국토부에 사업인정 협의 신청","용인시,플랫폼,시티,국토부,신청,사업,인정,협의,경기,용인시,중앙토지수용위원회,국토..."
54607,TRAIN_54607,지역,"용인시 이동읍 주민자치위원회, 저소득 20가구에 밑반찬 지원","주민자치위원회,용인시,이동읍,주민,자치,위원회,가구,밑반찬,지원,용인시,처인구,이동..."


In [ ]:
train_df['분류'].value_counts()

,count
분류,
지역,26950
경제:부동산,3454
사회:사건_사고,2568
경제:반도체,2318
사회:사회일반,1480
사회:교육_시험,995
정치:국회_정당,966
사회:의료_건강,950
경제:취업_창업,845


In [ ]:
train_df['키워드'][0]

'용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화재단,용인문화재단,제공,시민들,심신,여유,안정,인문학,콘서트,개최,재단,18일,큰어울마당,용인시,평생,학습관,인문학,콘서트,뮤지컬,이야기,인문학,콘서트,방송,매체,왕성,활동,뮤지컬,평론가,교수,원종원,순천향대,진행,이건명,김소향,김보경,유건우,배우들,정상급,뮤지컬,배우,라이브,무대,동시,만끽,공연,공연,홍지민,뮤지컬,배우,참여,라이브,무대,선사,영화,뮤지컬,합성어,무비컬,주제,맘마미아,드림걸즈,유명,작품,교수,위트,해설,예정,공연,초등학생,관람,초등학생,가능,티켓,가격,전석,공연,용인문화재단,누리집,확인'

In [ ]:
# 열의 중복을 제외한 값의 개수
train_df['제목'].nunique(), train_df['키워드'].nunique()

(52720, 54240)

In [ ]:
train_df['제목'].duplicated().sum(), train_df['키워드'].duplicated().sum()

(1889, 369)

In [ ]:
train_df.loc[train_df['제목'].duplicated(),'제목']

,제목
1285,“경력 연륜으로 중무장” 공공일자리 ‘내실화’
1925,"용인특례시의회 문화복지위원회, 행정사무감사 실시"
2418,백군기 용인시장 '자치분권 기대해' 챌린지 동참
2650,"용인중앙시장, 13일 '왁자지껄 봄 축제' 개최"
2667,[인사] 용인시
...,...
54511,"경기도, 지난해 '안전예방 핫라인' 통해 324건 무료 안전점검"
54516,학원가 집단감염 비상...2학기 전면등교 앞두고 '뇌관'
54544,"용인시, ‘반도체 필수 정제수’ 초순수 국산화 연구시설 유치 추진"
54570,"용인시, 코로나19 격리해제확인서 시 홈페이지서 발급"


In [ ]:
duplicated_list = list(train_df.loc[train_df['제목'].duplicated(),'제목'].unique())
duplicated_list

['“경력 연륜으로 중무장” 공공일자리 ‘내실화’',
 '용인특례시의회 문화복지위원회, 행정사무감사 실시',
 "백군기 용인시장 '자치분권 기대해' 챌린지 동참",
 "용인중앙시장, 13일 '왁자지껄 봄 축제' 개최",
 '[인사] 용인시',
 '용인시, 공예명장 문화상 후보자 공모',
 '경기도, 103개 평생학습마을 전수조사 9개 우수마을 선정',
 '용인에서 주차하던 승용차, 카페로 돌진...11명 부상',
 '[인사]용인시',
 '용인특례시의회, 제279회 임시회 폐회',
 '용인 반도체클러스터 전력공급시설 설치 계획 승인?고시',
 '용인시, 3차 성장관리계획 고시 따른 모니터링 진행',
 '이 시각 소방청 상황실',
 '백군기 용인시장, 자치단체 협력사업 경기도교육감 표창',
 '용인시의회, 제253회 임시회 개회',
 '산림청-현대백화점그룹, ‘탄소중립의 숲’ 조성',
 '모여라! 미래의 지도자들',
 '\'용인 경전철\' 주민들 일부 승소..."214억 청구해야"',
 '용인시, 민간포인트 와이페이로 쓸 수 있는 시스템 구축',
 '용인시, 제2용인테크노밸리 일반산업단지 보상계획 공고',
 '고교 진학상담 직원이 SAT 시험지 유출 정황...경찰, 압수수색',
 '용인특례시의회, 제272회 임시회 제2차 본회의 개최',
 "한국주택금융공사 경기동부지사, 용인시에 '둥지'",
 "[포토] 이예원, '힘차게 날아라~'",
 '용인시 처인구, 개발행위허가 기간 만료 사전 안내 서비스 도입',
 '용인시, 코로나19 안심식당 102곳 신규 지정',
 '코로나 확산에 “동해안 등 해맞이 명소 오지 마세요”',
 "용인시, '소규모 기업환경 개선사업' 참여 중소기업 모집",
 '국민 마라토너 이봉주, 용인시 홍보대사 위촉',
 '[부고] 이영민(용인시 행정과장)씨 장인상',
 '용인시, 장애인에 한국민속촌 에버랜드 무료 이용 지원',
 '"목사가 감금 폭행" 망상에 흉기 들고 교회 찾아간 40대 실형',
 '서커스 보면서 함박웃음...동심 속 세상이 된 도

In [ ]:
keyword_duplicated_list = []
for i in duplicated_list:
  if train_df.loc[train_df['제목'] == i, '키워드'].duplicated().sum() > 0:
    keyword_duplicated_list.append(i)

keyword_duplicated_list

['“경력 연륜으로 중무장” 공공일자리 ‘내실화’',
 '\'용인 경전철\' 주민들 일부 승소..."214억 청구해야"',
 '고교 진학상담 직원이 SAT 시험지 유출 정황...경찰, 압수수색',
 '코로나 확산에 “동해안 등 해맞이 명소 오지 마세요”',
 '서커스 보면서 함박웃음...동심 속 세상이 된 도심',
 '제네시스 G90 블랙 공개',
 '용인시, 정자교 사고 계기 827개 전체 교량 긴급 안전점검',
 '개그맨에서 한국을 대표하는 레이서로... 한민관, “올해 목표는 GT클래스 세 번째 시즌 챔피언!”',
 '대한적십자, 마스크 2,000만 장 방역 취약계층에 전달',
 '"겨울엔 강원도로 오세요" 화천 산천어 축제 개막',
 "'식용견' 낙인 도살장 개들, 해외에선 '환영' 국내선 '안락사 대상'",
 "경기도 이틀 연속 확진자 2백 명 초과...절반은 'n차 감염'",
 '용인시 “죽전 데이터센터, 전자파 우려 해소돼야 허가 검토”',
 '월급보다 물가가 더 올랐다 “작은 기업일수록 실질임금 적어”',
 '플라스틱 공장 대형마트서 화재 인명피해 없어',
 "택배 파업에 일부 배송 차질...택배노조 '상경 투쟁'",
 '승리, 특수폭행교사 혐의 추가 기소 "유인석이 조폭 불러 위협"',
 '코로나19 신규확진 75명 사흘연속 두 자릿수 유지',
 '영동고속도로 달리던 차량 4대 잇따라 부딪혀...2명 부상',
 'GTX-A 구성역 오는 29일 개통 수서까지 14분 동탄까지 7분',
 '그리운 일상으로 한발 더 ‘노인 시설’ 속속 운영 재개',
 '용인 매트리스 제조공장서 불...5시간 만에 진화',
 '[부고] 김민규(신세계그룹 커뮤니케이션본부장)씨 부친상',
 '영화관에서 즐기는 설 연휴...모처럼 활기 띤 극장가',
 '슈퍼레이스 GT1 우승 정경훈, 2위를 한 김동은은 경계의 대상!',
 '“이것이 춘천의 맛!” 원조 막국수 닭갈비 체험 축제 개막',
 '용인시, 환경보전 유공자 48명에 표창장 수여',
 '전북, ‘생활인구’도 적어 ‘지

In [ ]:
# 중복 된 제목의 키워드가 중복 된 제목 수
print(len(keyword_duplicated_list))

# 중복 된 제목 리스트 수
print(len(duplicated_list))

259
1460


In [ ]:
# '제목'이 keyword_duplicated_list에 있는 데이터에 대해서만 중복된 '키워드'를 처리
for title in keyword_duplicated_list:
    # 해당 '제목'에 대한 데이터 추출
    temp_df = train_df[train_df['제목'] == title]

    # '키워드'가 중복된 경우 중 첫 번째를 제외한 나머지를 제거
    train_df = train_df.drop(temp_df[temp_df.duplicated(subset=['키워드'], keep='first')].index)

# 최종 데이터프레임 출력
train_df

,ID,분류,제목,키워드
0,TRAIN_00000,문화:전시_공연,"용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최","용인문화재단,인문학,콘서트,뮤지컬,이야기,개최,인문학,콘서트,뮤지컬,이야기,용인문화..."
1,TRAIN_00001,지역,"용인 농촌테마파크, 7~8월 단체체험객 체험료 지원","용인,농촌,테마파크,단체,체험객,체험료,지원,15일,체험일,기준,용인시통합예약사이트..."
2,TRAIN_00002,지역,"용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장","용인시,노후,주택,에너지,성능,개선,신청,연장,용인시청,용인시,노후,건축물,환경친화..."
3,TRAIN_00003,지역,"수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대","수원,용인,고양시,특례시,지정,도시경쟁력,증가,경기,도내,인구,수원,고양,용인시,특..."
4,TRAIN_00004,국제,"용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할","용인시,스페인,미국,국제,명예,자문관,위촉,역할,대외,홍보,지원,용인시,권태면,주코..."
...,...,...,...,...
54604,TRAIN_54604,국제,"용인 아파트서 30대 여성, 아들 딸 함께 추락 ""극단 선택 추정""","용인,아파트,여성,아들,추락,극단,선택,추정,경찰,현장,유서,경위,조사,처인구,경기..."
54605,TRAIN_54605,사회:교육_시험,"용인시, '위탁부모 보수교육' 실시","용인시,위탁,부모,보수,교육,용인시,경기,남부,가정,위탁,지원,센터,시청,컨퍼런스,..."
54606,TRAIN_54606,지역,"용인시, '플랫폼 시티' 국토부에 사업인정 협의 신청","용인시,플랫폼,시티,국토부,신청,사업,인정,협의,경기,용인시,중앙토지수용위원회,국토..."
54607,TRAIN_54607,지역,"용인시 이동읍 주민자치위원회, 저소득 20가구에 밑반찬 지원","주민자치위원회,용인시,이동읍,주민,자치,위원회,가구,밑반찬,지원,용인시,처인구,이동..."


In [ ]:
train_df.loc[train_df['키워드'].duplicated(), '분류'].value_counts()

,count
분류,
지역,19
스포츠:골프,14
스포츠:올림픽_아시안게임,11
경제:부동산,5
문화:음악,4
경제:금융_재테크,2
사회:날씨,2
사회:사회일반,2
국제,2


In [ ]:
train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']
train_df['제목_키워드'].duplicated().sum()

0

In [ ]:
# 한글을 제외한 나머지 문자들 처리
train_df['제목_키워드'] = train_df['제목_키워드'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]", " ", regex=True)
test_df['제목_키워드'] = test_df['제목_키워드'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]", " ", regex=True)
train_df['제목_키워드'][0]

'용인문화재단  인문학 콘서트  당신이 모르는 뮤지컬 이야기   개최 용인문화재단 인문학 콘서트 뮤지컬 이야기 개최 인문학 콘서트 뮤지컬 이야기 용인문화재단 용인문화재단 제공 시민들 심신 여유 안정 인문학 콘서트 개최 재단   일 큰어울마당 용인시 평생 학습관 인문학 콘서트 뮤지컬 이야기 인문학 콘서트 방송 매체 왕성 활동 뮤지컬 평론가 교수 원종원 순천향대 진행 이건명 김소향 김보경 유건우 배우들 정상급 뮤지컬 배우 라이브 무대 동시 만끽 공연 공연 홍지민 뮤지컬 배우 참여 라이브 무대 선사 영화 뮤지컬 합성어 무비컬 주제 맘마미아 드림걸즈 유명 작품 교수 위트 해설 예정 공연 초등학생 관람 초등학생 가능 티켓 가격 전석 공연 용인문화재단 누리집 확인'

In [ ]:
stopwords = ["가", "가까스로", "가령", "각", "각각", "각자", "각종", "갖고말하자면", "같다", "같이", "개의치않고", "거니와", "거바", "거의", "것", "것과 같이", "것들", "게다가", "게우다", "겨우", "견지에서", "결과에 이르다", "결국", "결론을 낼 수 있다", "겸사겸사", "고려하면", "고로", "곧", "공동으로", "과", "과연", "관계가 있다", "관계없이", "관련이 있다", "관하여", "관한", "관해서는", "구", "구체적으로", "구토하다", "그", "그들", "그때", "그래", "그래도", "그래서", "그러나", "그러니","그러니까", "그러면", "그러므로", "그러한즉", "그런 까닭에", "그런데", "그런즉", "그럼","그럼에도 불구하고", "그렇게 함으로써", "그렇지", "그렇지 않다면", "그렇지 않으면", "그렇지만", "그렇지않으면", "그리고", "그리하여", "그만이다", "그에 따르는", "그위에", "그저", "그중에서", "그치지 않다", "근거로", "근거하여", "기대여", "기점으로", "기준으로", "기타", "까닭으로", "까악", "까지", "까지 미치다", "까지도", "꽈당", "끙끙", "끼익", "나", "나머지는", "남들", "남짓", "너", "너희", "너희들", "네", "넷", "년", "논하지 않다", "놀라다", "누가 알겠는가", "누구", "다른", "다른 방면으로", "다만", "다섯", "다소", "다수", "다시 말하자면", "다시말하면", "다음", "다음에", "다음으로", "단지", "답다", "당신", "당장", "대로 하다", "대하면", "대하여", "대해 말하자면", "대해서", "댕그", "더구나", "더군다나", "더라도", "더불어", "더욱더", "더욱이는", "도달하다",     "도착하다", "동시에", "동안", "된바에야", "된이상", "두번째로", "둘", "둥둥", "뒤따라", "뒤이어", "든간에", "들", "등", "등등", "딩동", "따라", "따라서", "따위", "따지지 않다", "딱", "때", "때가 되어", "때문에", "또", "또한", "뚝뚝", "라 해도", "령", "로", "로 인하여", "로부터", "로써", "륙", "를", "마음대로", "마저", "마저도", "마치", "막론하고", "만 못하다", "만약", "만약에", "만은 아니다", "만이 아니다", "만일", "만큼", "말하자면", "말할것도 없고", "매", "매번", "메쓰겁다", "몇", "모", "모두", "무렵", "무릎쓰고", "무슨", "무엇", "무엇때문에", "물론", "및", "바꾸어말하면", "바꾸어말하자면", "바꾸어서 말하면", "바꾸어서 한다면", "바꿔 말하면", "바로", "바와같이", "밖에 안된다", "반대로", "반대로 말하자면", "반드시", "버금", "보는데서", "보다더", "보드득", "본대로", "봐", "봐라", "부류의 사람들", "부터", "불구하고", "불문하고", "붕붕", "비걱거리다", "비교적", "비길수 없다", "비로소", "비록", "비슷하다", "비추어 보아", "비하면", "뿐만 아니라", "뿐만아니라", "뿐이다", "삐걱", "삐걱거리다", "사", "삼", "상대적으로 말하자면", "생각한대로", "설령", "설마", "설사", "셋", "소생", "소인", "솨", "쉿", "습니까", "습니다", "시각", "시간", "시작하여", "시초에", "시키다", "실로", "심지어", "아", "아니", "아니나다를가", "아니라면", "아니면", "아니었다면", "아래윗", "아무거나", "아무도", "아야", "아울러", "아이", "아이고", "아이구", "아이야", "아이쿠", "아하", "아홉", "안 그러면", "않기 위하여", "않기 위해서", "알 수 있다", "알았어", "앗", "앞에서", "앞의것", "야", "약간", "양자", "어", "어기여차", "어느", "어느 년도", "어느것", "어느곳", "어느때", "어느쪽", "어느해", "어디", "어때", "어떠한", "어떤", "어떤것", "어떤것들", "어떻게", "어떻해", "어이", "어째서", "어쨋든", "어쩔수 없다", "어찌", "어찌됏든", "어찌됏어", "어찌하든지", "어찌하여", "언제", "언젠가", "얼마", "얼마 안 되는 것", "얼마간", "얼마나", "얼마든지", "얼마만큼", "얼마큼", "엉엉", "에", "에 가서", "에 달려 있다", "에 대해", "에 있다", "에 한하다", "에게", "에서", "여", "여기", "여덟", "여러분", "여보시오", "여부", "여섯", "여전히", "여차", "연관되다", "연이서", "영", "영차", "옆사람", "예", "예를 들면", "예를 들자면", "예컨대", "예하면", "오", "오로지", "오르다", "오자마자", "오직", "오호", "오히려", "와", "와 같은 사람들", "와르르", "와아", "왜", "왜냐하면", "외에도", "요만큼", "요만한 것", "요만한걸", "요컨대", "우르르", "우리", "우리들", "우선", "우에 종합한것과같이", "운운", "월", "위에서 서술한바와같이", "위하여", "위해서", "윙윙", "육", "으로", "으로 인하여", "으로서", "으로써", "을", "응", "응당", "의", "의거하여", "의지하여", "의해", "의해되다", "의해서", "이", "이 되다", "이 때문에", "이 밖에", "이 외에", "이 정도의", "이것", "이곳", "이때", "이라면", "이래", "이러이러하다", "이러한", "이런", "이럴정도로", "이렇게 많은 것", "이렇게되면", "이렇게말하자면", "이렇구나", "이로 인하여", "이르기까지", "이리하여", "이만큼", "이번", "이봐", "이상", "이어서", "이었다", "이와 같다", "이와 같은", "이와 반대로", "이와같다면", "이외에도", "이용하여", "이유만으로", "이젠", "이지만", "이쪽", "이천구", "이천육", "이천칠", "이천팔", "인 듯하다", "인젠", "일", "일것이다", "일곱", "일단", "일때", "일반적으로", "일지라도", "임에 틀림없다", "입각하여", "입장에서", "잇따라", "있다", "자", "자기", "자기집", "자마자", "자신", "잠깐", "잠시", "저", "저것", "저것만큼", "저기", "저쪽", "저희", "전부", "전자", "전후", "점에서 보아", "정도에 이르다", "제", "제각기", "제외하고", "조금", "조차", "조차도", "졸졸", "좀", "좋아", "좍좍", "주룩주룩", "주저하지 않고", "줄은 몰랏다", "줄은모른다", "중에서", "중의하나", "즈음하여", "즉", "즉시", "지든지", "지만", "지말고", "진짜로", "쪽으로", "차라리", "참", "참나", "첫번째로", "쳇", "총적으로", "총적으로 말하면", "총적으로 보면", "칠", "콸콸", "쾅쾅", "쿵", "타다", "타인", "탕탕", "토하다", "통하여", "툭", "퉤", "틈타", "팍", "팔", "퍽", "펄렁", "하", "하게될것이다", "하게하다", "하겠는가", "하고 있다", "하고있었다", "하곤하였다", "하구나", "하기 때문에", "하기 위하여", "하기는한데", "하기만 하면", "하기보다는", "하기에", "하나", "하느니", "하는 김에", "하는 편이 낫다", "하는것도", "하는것만 못하다", "하는것이 낫다", "하는바", "하더라도", "하도다", "하도록시키다", "하도록하다", "하든지", "하려고하다", "하마터면", "하면 할수록", "하면된다", "하면서", "하물며", "하여금", "하여야", "하자마자", "하지 않는다면", "하지 않도록", "하지마", "하지마라", "하지만", "하하", "한 까닭에", "한 이유는", "한 후", "한다면", "한다면 몰라도", "한데", "한마디", "한적이있다", "한켠으로는", "한항목", "할 따름이다", "할 생각이다", "할 줄 안다", "할 지경이다", "할 힘이 있다", "할때", "할만하다", "할망정", "할뿐", "할수있다", "할수있어", "할줄알다", "할지라도", "할지언정", "함께", "해도된다", "해도좋다", "해봐요", "해서는 안된다", "해야한다", "해요", "했어요", "향하다", "향하여", "향해서", "허", "허걱", "허허", "헉", "헉헉", "헐떡헐떡", "형식으로 쓰여", "혹시", "혹은", "혼자", "훨씬", "휘익", "휴", "흐흐", "흥", "힘입어"]


['가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',
 '더구나'

In [ ]:
from konlpy.tag import Okt

# Okt 형태소 분석기
okt = Okt()

# 형태소 분석 후 불용어 제거
def remove_stopwords_with_okt(sentence, stopwords):
    tokenized_sentence = okt.morphs(sentence)  # 형태소 분석
    return ' '.join([word for word in tokenized_sentence if word not in stopwords])

clean_train_kewords = []
for keword in train_df['제목_키워드']:
    clean_train_kewords.append(remove_stopwords_with_okt(keword,stopwords))

clean_test_kewords= []
for keword in test_df['제목_키워드']:
    clean_test_kewords.append(remove_stopwords_with_okt(keword,stopwords))

In [ ]:
train_df1 = pd.DataFrame({'제목_키워드': clean_train_kewords, '분류': train_df['분류']})
test_df1 = pd.DataFrame({'제목_키워드': clean_test_kewords})

In [ ]:
train_df1['제목_키워드'] = train_df1['제목_키워드'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))
test_df1['제목_키워드'] = test_df1['제목_키워드'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))

In [ ]:
train_df1['제목_키워드'][0]

'용인 문화재단 인문학 콘서트 모르는 뮤지컬 이야기 개최 용인 문화재단 인문학 콘서트 뮤지컬 이야기 개최 인문학 콘서트 뮤지컬 이야기 용인 문화재단 용인 문화재단 제공 시민 심신 여유 안정 인문학 콘서트 개최 재단 큰어울 마당 용인시 평생 습관 인문학 콘서트 뮤지컬 이야기 인문학 콘서트 방송 매체 왕성 활동 뮤지컬 평론가 교수 원종원 순천향대 진행 이건명 김소향 김보경 유건우 배우 정상 뮤지컬 배우 라이브 무대 동시 만끽 공연 공연 홍지민 뮤지컬 배우 참여 라이브 무대 선사 영화 뮤지컬 합성어 무비 주제 맘마미아 드림걸즈 유명 작품 교수 위트 해설 예정 공연 초등학생 관람 초등학생 가능 티켓 가격 공연 용인 문화재단 누리집 확인'

In [ ]:
train_df1

,제목_키워드,분류
0,용인 문화재단 인문학 콘서트 모르는 뮤지컬 이야기 개최 용인 문화재단 인문학 콘서트...,문화:전시_공연
1,용인 농촌 테마 파크 단체 체험 체험 지원 용인 농촌 테마 파크 단체 체험 체험 지...,지역
2,용인시 노후 주택 에너지 성능 개선 신청 연장 용인시 노후 주택 에너지 성능 개선 ...,지역
3,수원 용인 고양시 특례시 지정 도시 경쟁력 증가 기대 수원 용인 고양시 특례시 지정...,지역
4,용인시 스페인 미국 국제 명예 문관 위촉 대외 홍보 지원 역할 용인시 스페인 미국 ...,국제
...,...,...
54604,용인 아파트 여성 아들 추락 극단 선택 추정 용인 아파트 여성 아들 추락 극단 선택...,국제
54605,용인시 위탁 부모 보수 교육 실시 용인시 위탁 부모 보수 교육 용인시 경기 남부 가...,사회:교육_시험
54606,용인시 플랫폼 시티 국토부 사업인정 협의 신청 용인시 플랫폼 시티 국토부 신청 사업...,지역
54607,용인시 동읍 주민 자치 위원회 소득 가구 밑반찬 지원 주민 자치 위원회 용인시 동읍...,지역


In [ ]:
test_df1.to_csv('test_df_jiwon.csv',encoding='utf-8-sig')
train_df1.to_csv('train_df_jiwon.csv',encoding='utf-8-sig')